In [1]:
import backtrader as bt
import backtrader.analyzers as btanalyzers
from GetData import getData
import numpy as np
import matplotlib as plt
import math

SyntaxError: invalid syntax (__init__.py, line 239)

In [ ]:
%matplotlib inline

In [ ]:
#date = '2022-05-16'
symbol = 'NG=F'
interval = '1d'
period = '1y'

In [ ]:
#date = '2022-05-16'
symbol = 'NG=F'
interval = '1m'
period = '1wk'

In [ ]:
getdata = getData()
rawData = getdata.getHistoryYfinance(symbol=symbol, interval=interval, period=period)

[*********************100%***********************]  1 of 1 completed


In [ ]:
rawData.count()

Open         3643
High         3643
Low          3643
Close        3643
Adj Close    3643
Volume       3643
dtype: int64

In [ ]:
class GoldenCross(bt.Strategy):
    
    # list of parameters which are configurable for the strategy
    params = dict(
        pfast=8,  
        pslow=70,
        pOrder_percentage=0.95,
        pTicker='SPY'  
    )

    def __init__(self):
        self.fast_moving_average = bt.ind.SMA(self.data.close, period=self.p.pfast, plotname= str(self.p.pfast) + ' Moving average')  # fast moving average
        self.slow_moving_average = bt.ind.SMA(self.data.close, period=self.p.pslow, plotname= str(self.p.pslow) + ' Moving average')  # slow moving average
        self.crossOver = bt.ind.CrossOver(self.fast_moving_average, self.slow_moving_average)  # crossover signal
                    
    def next(self):
        if self.position.size == 0:
            if self.crossOver > 0:  # GoldenCross
                amount_to_invest = (self.p.pOrder_percentage * self.broker.cash)
                self.size = math.floor(amount_to_invest / self.data.close[0])

                print("Buy {} shares of {} at {}".format(self.size, self.p.pTicker, self.data.close[0]))
                self.buy(size=self.size)

        if self.position.size > 0:
            if self.crossOver < 0:  # 
                print("Sell {} shares of {} at {}".format(self.size, self.p.pTicker, self.data.close[0]))
                self.close()

In [ ]:
cerebro = bt.Cerebro()  # create a "Cerebro" engine instance

# Create a data feed
data = bt.feeds.PandasData(dataname=rawData)

cerebro.adddata(data)  # Add the data feed
cerebro.broker.set_cash(1000)
cerebro.addstrategy(GoldenCross)  # Add the trading strategy
cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='mysharpe')
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
thestrats = cerebro.run()
thestrat = thestrats[0]

Buy 117 shares of SPY at 8.07800006866455
Sell 117 shares of SPY at 8.08899974822998
Buy 117 shares of SPY at 8.102999687194824
Sell 117 shares of SPY at 8.088000297546387
Buy 116 shares of SPY at 8.131999969482422
Sell 116 shares of SPY at 8.128999710083008
Buy 116 shares of SPY at 8.135000228881836
Sell 116 shares of SPY at 8.111000061035156
Buy 116 shares of SPY at 8.13599967956543
Sell 116 shares of SPY at 8.119999885559082
Buy 118 shares of SPY at 7.9770002365112305
Sell 118 shares of SPY at 7.939000129699707
Buy 118 shares of SPY at 7.949999809265137
Sell 118 shares of SPY at 7.923999786376953
Buy 117 shares of SPY at 7.995999813079834
Sell 117 shares of SPY at 8.531999588012695
Buy 115 shares of SPY at 8.588000297546387
Sell 115 shares of SPY at 8.748000144958496
Buy 115 shares of SPY at 8.782999992370605
Sell 115 shares of SPY at 8.781999588012695
Buy 115 shares of SPY at 8.767000198364258
Sell 115 shares of SPY at 8.769000053405762
Buy 115 shares of SPY at 8.744999885559082
Se

In [ ]:
print('Final value', cerebro.broker.getvalue())
print('Sharpe Ratio:', thestrat.analyzers.mysharpe.get_analysis())

Final value 1054.405288696289
Sharpe Ratio: OrderedDict([('sharperatio', None)])


In [ ]:
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False
plt.rcParams['figure.figsize']=[18, 16]
plt.rcParams['figure.dpi']=200
plt.rcParams['figure.facecolor']='w'
plt.rcParams['figure.edgecolor']='k'
cerebro.plot(height= 30, iplot= False)
#cerebro.plot()

[[<Figure size 3600x3200 with 5 Axes>]]

: 